In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from train import Trainer

run = Trainer()
run.train()

  0%|          | 0/9982 [00:00<?, ?it/s]

In [1]:
import torch
torch.backends.mps.is_built()

True

In [93]:
import torch.nn as nn


class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding='same', bias=False),
            nn.ELU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding='same', bias=False),
            nn.BatchNorm2d(128),
            nn.ELU(),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding='same', bias=False),
            nn.BatchNorm2d(256),
            nn.ELU(),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding='same', bias=False),
            nn.BatchNorm2d(512),
            nn.ELU(),
            nn.Conv2d(in_channels=512, out_channels=1, kernel_size=1, bias=False)
        )
        self.act = nn.Sigmoid()

    def forward(self, input):
        out = self.classifier(input)
        out = out.view((input.shape[0], -1))
        out = self.act(out)
        return out


class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(in_channels=36, out_channels=512, kernel_size=4, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=4, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels=128, out_channels=1, kernel_size=4, bias=False),
            nn.Tanh()
        )
        self.output_linear = nn.Linear(in_features=169, out_features=150)

    def forward(self, input):
        input = input.unsqueeze(-1).unsqueeze(-1)
        out = self.gen(input)
        out = out.view((-1, 169))
        out = self.output_linear(out)
        out = out.view((-1, 1, 10, 15))
        return out

In [94]:
device = torch.device("mps")

gen = Generator().to(device)
disc = Discriminator().to(device)

t = torch.randn((10, 36), device = device)

out = gen(t).detach()

logits = disc(out)

logits.shape

torch.Size([10, 150])